# Make a TinyML Lite Model for Arduino


In [1]:
## Library import
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import random

from warnings import simplefilter
simplefilter('ignore')

## Model
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()

### Set model file paths

In [2]:
## Environment setup
COLAB = True # False: local environment
PROJECT_DIR = "/content/drive/MyDrive/Project/CS565_IoT/"

In [3]:
if COLAB:
  ## Linkage Google Drive
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
MODELS_DIR = PROJECT_DIR + "Models/"
MODEL_TF = MODELS_DIR + 'model.keras'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

## Preprocessing for arduino

Constants for preprocessing were chosen during training.

In [5]:
median = 507.0
median_absolute_deviation = 3.0
mean = 508.96049074074074
std = 7.95560041149137

In [6]:
### Outlier removal
'''
If value of |z-score| > 3.5 then, remove
'''

def outliers_modified_z(data, threshold=3.5):
    global median, median_absolute_deviation

    data = data.copy()

    modified_z_scores = 0.6745 * (data - median) / median_absolute_deviation
    outliers = np.abs(modified_z_scores) > threshold
    data[outliers] = median
    return data

### Normalization
'''
Apply Standard Normalization
'''

def standard_scaler(data):
    global mean, std
    scaled_data = (data - mean) / std
    return scaled_data

## Convert the model to TF Lite

### Load model

In [7]:
model = tf.keras.models.load_model(MODEL_TF)

### Convert the model to TF Lite

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model_tflite = converter.convert()

open(MODEL_TFLITE, "wb").write(model_tflite)

104164

In [9]:
# Helper class for TF Lite Model
class MyTFLiteModel:
  # global class_num

  def __init__(self, model_path):
    self.interpreter = tf.lite.Interpreter(model_path)
    self.interpreter.allocate_tensors()

    interpreter_input = self.interpreter.tensor(self.interpreter.get_input_details()[0]["index"])
    interpreter_output = self.interpreter.tensor(self.interpreter.get_output_details()[0]["index"])

  def predict(self, X_test):
    input_details = self.interpreter.get_input_details()[0]
    output_details = self.interpreter.get_output_details()[0]
    interpreter_predictions = np.empty((len(X_test), class_num))

    for i in range(len(X_test)):
      self.interpreter.set_tensor(input_details["index"], X_test[i].reshape(1, -1, 1))
      self.interpreter.invoke()
      interpreter_predictions[i] = self.interpreter.get_tensor(output_details["index"])[0]

    return interpreter_predictions

## Convert the model to C code

In [11]:
# Convert the model to a C source file
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

# Update var names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
## Replace {REPLACE_TEXT} with 'g_model'
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

# Compare Arduino output with original

#### Load the arduino output



In [35]:
ARDUINO_DIR = PROJECT_DIR + "Tests/"
ARDUINO_OUTPUT_FILE = ARDUINO_DIR + "arduino.txt"
INFERENCE_FILE = ARDUINO_DIR + "inferences.csv"
PPG_FILE = ARDUINO_DIR + "ppg.csv"

cut_sample = 200
cut_sample_collection = 240
class_num = 2


CMD_MAKE_INFERENCE_FILE = """cat %s | grep Inference | awk '{print $2 ", " $3}' > %s""" % (ARDUINO_OUTPUT_FILE, INFERENCE_FILE)
!{CMD_MAKE_INFERENCE_FILE}
!sed '/Inference/d' {ARDUINO_OUTPUT_FILE} > {PPG_FILE}
# Remove first two lines, which are not ppg values
!sed -i '1d' {PPG_FILE}
!sed -i '1d' {PPG_FILE}

In [24]:
arduino_ppg_values = pd.read_csv(PPG_FILE, names=["timestamp", "ppg"])
arduino_inferences = pd.read_csv(INFERENCE_FILE, names=["0", "1"])

In [25]:
!head {INFERENCE_FILE}

0.000024, 0.999924
0.135200, 0.770444
0.295312, 0.686475
0.648624, 0.415240
0.003676, 0.995903
0.002450, 0.995505
0.474002, 0.369889
0.032977, 0.960106
0.999811, 0.000798


In [26]:
# Check the validity

assert len(arduino_inferences) * cut_sample_collection == len(arduino_ppg_values)
print(len(arduino_inferences))

9


### Preprocess the ppg data collected by arduino

### Predict by original, TF Lite, and arduino.



#### Load the TF Lite model

In [20]:
model_tflite = MyTFLiteModel(MODEL_TFLITE)

In [33]:
# Preprocess as if it is running in arduino

col = 'tiny'
arduino_ppg_values[col] = outliers_modified_z(arduino_ppg_values['ppg'])
arduino_ppg_values[col] = standard_scaler(arduino_ppg_values[col])
X_test = arduino_ppg_values[col].values.reshape(-1,cut_sample_collection)[:,:cut_sample].astype(np.float32)

In [36]:
y_predict = model.predict(X_test)
y_predict_lite = model_tflite.predict(X_test)

if np.abs((y_predict - y_predict_lite).max().max()) < 10**-6:
  print("TF Lite model made same prediction with the original model "
        "up to 6 decimal places.")

if np.abs((y_predict - arduino_inferences).max().max()) < 10**-6:
  print("Arduino made same prediction with the original model "
        "up to 6 decimal places.")
  print()

print("Arduino predictions")
arduino_inferences.head()


1/1 [==============================] - 0s 20ms/step
TF Lite model made same prediction with the original model up to 6 decimal places.
Arduino predictions


,0,1
0,0.000024,0.999924
1,0.135200,0.770444
2,0.295312,0.686475
3,0.648624,0.415240
4,0.003676,0.995903
